<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/introOptimization/blob/main/Intro_To_Optimization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Introduction to Optimization for Decision Making


In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="1" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 1: Maximization/Minimization

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [7]:
pip show glpk pulp

Name: PuLP
Version: 2.8.0
Summary: PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.
Home-page: https://github.com/coin-or/pulp
Author: J.S. Roy and S.A. Mitchell and F. Peschiera
Author-email: pulp@stuartmitchell.com
License: 
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [19]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [21]:
# how much gas?
Gas = pp.LpVariable(name="Gas",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

# how much oil?
Oil = pp.LpVariable(name="Oil",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__:

In [23]:
GasCoeff=1.9
OilCoeff=1.5
obj_func = GasCoeff*Gas + OilCoeff*Oil

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [25]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Gas Constraint',   # just the name
                    e= 1*Gas - 2*Oil, rhs=0, # linear combination of constraint and rhs
                    sense=pp.LpConstraintGE) # 'rule' >= 0 (LpConstraintGE)
C2= pp.LpConstraint(name='Oil Constraint',
                    e= 1*Oil, rhs=3000000,
                    sense=pp.LpConstraintGE) # 'rule' >= 3000000 (LpConstraintGE)
C3= pp.LpConstraint(name='Demand Constraint',
                    e= 1*Gas, rhs=6400000,
                    sense=pp.LpConstraintLE, )# 'rule' <= 6400000 (LpConstraintLE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [27]:
model += obj_func
model += C1
model += C2
model += C3

6. **Solve the MODEL**: Notice we are not using the _default solver_, we are explicitly usig **COIN_CMD**:

In [29]:
solver_list = pp.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [47]:
solverToUse = pp.COIN_CMD(path="D:/2024-1/Herramientas cuantitativas/Tarea 5/bin/cbc.exe", msg=False) 
model.solve(solver=solverToUse);

You can create a summary like this:

In [51]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Solution': 16960000.0,
 'Gas': 6400000.0,
 'Oil': 3200000.0}

In [53]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Solution,Gas,Oil
Model Status,,,
Optimal,"16,960,000.0","6,400,000.0","3,200,000.0"


<div class="alert-success">

<strong>Exercise: The diet problem</strong>

In [55]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 2: Multicriteria Decision-Making

In [57]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

The following code requires the folder **ahp**, which you have to download from [here](https://github.com/gprzy/ahp), or from [here](https://github.com/SocialAnalytics-StrategicIntelligence/introOptimization/raw/main/ahp.zip). Make sure the folder with the two **.py** files are in this notebook. You need pandas and numpy previously installed.

In [59]:
# calling the function
from ahp.ahp import AHP

I have previously prepared the pairwise comparisons:

In [61]:
import pandas as pd

pairwise_age=pd.read_excel("ahp_tables.xlsx",sheet_name='age', index_col=0)
pairwise_experience=pd.read_excel("ahp_tables.xlsx",sheet_name='experience', index_col=0)
pairwise_education=pd.read_excel("ahp_tables.xlsx",sheet_name='education', index_col=0)
pairwise_charisma=pd.read_excel("ahp_tables.xlsx",sheet_name='charisma', index_col=0)
pairwise_criteria=pd.read_excel("ahp_tables.xlsx",sheet_name='criteria', index_col=0)

Open the *pairwise_criteria* to check the order of columns:

In [63]:
pairwise_criteria

,Experience,Education,Charisma,Age
Experience,1.000000,4.000000,3.000000,7
Education,0.250000,1.000000,0.333333,3
Charisma,0.333333,3.000000,1.000000,5
Age,0.142857,0.333333,0.200000,1


Use the same order here:

In [65]:
# follow the same order as the criteria
data = [pairwise_experience.values.tolist(),
        pairwise_education.values.tolist(),
        pairwise_charisma.values.tolist(),
        pairwise_age.values.tolist(),
        pairwise_criteria.values.tolist()]

Time to fit the AHP model:

In [67]:
ahp = AHP()
ahp.fit(data)

You get the solution:

In [69]:
Solution=pd.DataFrame(index=pairwise_charisma.index)
Solution['Score']=ahp.classificate()
Solution['Ranking']=Solution.Score.rank(ascending=False)
Solution

,Score,Ranking
Tom,0.359067,2.0
Dick,0.488307,1.0
Harry,0.152626,3.0


<div class="alert-success">

<strong>Exercise: Choosing a country for a Master Program</strong>

- Join 3 or 4 people from this course (or other friends if needed).
- If you have the criteria: cost of living, language difficulty, possibilities to get a job in that country
- If you have the alternatives: Brazil, Spain, USA, England, France
- Create an AHP model and get the ranking.

You can follow this [example](https://en.wikipedia.org/wiki/Analytic_hierarchy_process_%E2%80%93_leader_example).
If you have a better idea, you can use it instead.

# Ejercicio ficticio

El ejercicio ficticio que he elaborada trata de un proceso de selección en donde tres candidatos son finalistas: Pedro, Miguel y Aldo. Estos candidatos estan luchando por quién ira de intercambio a otro país en base a tres criterios: nivel económico, nivel de lenguaje (inglés) y el promedio final en sus calificaciones.

In [75]:
import pandas as pd

pairwise_age_exercise=pd.read_excel("ahp_tables_exercise.xlsx",sheet_name='age', index_col=0)
pairwise_economy_exercise=pd.read_excel("ahp_tables_exercise.xlsx",sheet_name='economy', index_col=0)
pairwise_language_exercise=pd.read_excel("ahp_tables_exercise.xlsx",sheet_name='language', index_col=0)
pairwise_score_exercise=pd.read_excel("ahp_tables_exercise.xlsx",sheet_name='score', index_col=0)
pairwise_criteria_exercise=pd.read_excel("ahp_tables_exercise.xlsx",sheet_name='criteria', index_col=0)

In [77]:
pairwise_criteria_exercise

,Economy,Language,Score,Age
Economy,1.000000,7.000000,3.000000,8
Language,0.142857,1.000000,0.333333,3
Score,0.333333,3.000000,1.000000,5
Age,0.125000,0.333333,0.200000,1


In [79]:
data_exercise = [pairwise_economy_exercise.values.tolist(),
        pairwise_language_exercise.values.tolist(),
        pairwise_score_exercise.values.tolist(),
        pairwise_age_exercise.values.tolist(),
        pairwise_criteria_exercise.values.tolist()]

In [97]:
ahp = AHP()
ahp.fit(data_exercise)

In [99]:
Solution_exercise=pd.DataFrame(index=pairwise_economy_exercise.index)
Solution_exercise['Score']=ahp.classificate()
Solution_exercise['Ranking']=Solution_exercise.Score.rank(ascending=False)
Solution_exercise

,Score,Ranking
Pedro,0.273522,2.0
Miguel,0.483984,1.0
Aldo,0.242494,3.0
